# Querying

## Install toolkit

Run the command below to install the graphrag-toolkit. If you've already installed the toolkit, you don't need to install it again.

In [ ]:
!pip install https://github.com/awslabs/graphrag-toolkit/releases/latest/download/graphrag-toolkit.zip

### TraversalBasedRetriever

See [TraversalBasedRetriever](https://github.com/awslabs/graphrag-toolkit/blob/main/docs/querying.md#traversalbasedretriever).

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit import set_logging_config
from graphrag_toolkit import LexicalGraphQueryEngine, format_source
from graphrag_toolkit.storage import GraphStoreFactory
from graphrag_toolkit.storage import VectorStoreFactory

import nest_asyncio
nest_asyncio.apply()

set_logging_config('INFO', ['graphrag_toolkit.storage'])

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

query_engine = LexicalGraphQueryEngine.for_traversal_based_search(
    graph_store, 
    vector_store,
    post_processors=[
        format_source('url')
    ]
)

response = query_engine.query("What are the differences between Neptune Database and Neptune Analytics?")

print(response.response)

In [ ]:
for n in response.source_nodes:
    print(n.text)

In [ ]:
for n in response.source_nodes:
    print(n.metadata)

#### Set subretriever

In the example below, the `TraversalBasedRetriever` is configured with a `ChunkBasedSearch` subretriever.

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit import LexicalGraphQueryEngine, format_source
from graphrag_toolkit.storage import GraphStoreFactory
from graphrag_toolkit.storage import VectorStoreFactory
from graphrag_toolkit.retrieval.retrievers import ChunkBasedSearch

import nest_asyncio
nest_asyncio.apply()

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

query_engine = LexicalGraphQueryEngine.for_traversal_based_search(
    graph_store, 
    vector_store,
    retrievers=[ChunkBasedSearch],
    post_processors=[
        format_source('url')
    ]
)

response = query_engine.query("What are the differences between Neptune Database and Neptune Analytics?")

print(response.response)

### SemanticGuidedRetriever

See [SemanticGuidedRetriever](https://github.com/awslabs/graphrag-toolkit/blob/main/docs/querying.md#semanticguidedretriever).

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit import LexicalGraphQueryEngine
from graphrag_toolkit.storage import GraphStoreFactory
from graphrag_toolkit.storage import VectorStoreFactory

import nest_asyncio
nest_asyncio.apply()

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

query_engine = LexicalGraphQueryEngine.for_semantic_guided_search(
    graph_store, 
    vector_store
)

response = query_engine.query("What are the differences between Neptune Database and Neptune Analytics?")

print(response.response)

#### Set subretrievers

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit import LexicalGraphQueryEngine, format_source
from graphrag_toolkit.storage import GraphStoreFactory
from graphrag_toolkit.storage import VectorStoreFactory
from graphrag_toolkit.retrieval.retrievers import StatementCosineSimilaritySearch, KeywordRankingSearch, SemanticBeamGraphSearch

import nest_asyncio
nest_asyncio.apply()

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

query_engine = LexicalGraphQueryEngine.for_semantic_guided_search(
    graph_store, 
    vector_store,
    retrievers=[
        StatementCosineSimilaritySearch, 
        KeywordRankingSearch, 
        SemanticBeamGraphSearch
    ]
)

response = query_engine.query("What are the differences between Neptune Database and Neptune Analytics?")

print(response.response)

#### Reranking beam search (CPU)

The example below uses a `SentenceReranker` with a `RerankingBeamGraphSearch` to rerank statements while conducting the beam search.

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit import LexicalGraphQueryEngine
from graphrag_toolkit.storage import GraphStoreFactory
from graphrag_toolkit.storage import VectorStoreFactory
from graphrag_toolkit.retrieval.retrievers import RerankingBeamGraphSearch, StatementCosineSimilaritySearch, KeywordRankingSearch
from graphrag_toolkit.retrieval.post_processors import SentenceReranker

import nest_asyncio
nest_asyncio.apply()

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

cosine_retriever = StatementCosineSimilaritySearch(
    vector_store=vector_store,
    graph_store=graph_store,
    top_k=50
)

keyword_retriever = KeywordRankingSearch(
    vector_store=vector_store,
    graph_store=graph_store,
    max_keywords=10
)

reranker = SentenceReranker(
    batch_size=128
)

beam_retriever = RerankingBeamGraphSearch(
    vector_store=vector_store,
    graph_store=graph_store,
    reranker=reranker,
    initial_retrievers=[cosine_retriever, keyword_retriever],
    max_depth=8,
    beam_width=100
)

query_engine = LexicalGraphQueryEngine.for_semantic_guided_search(
    graph_store, 
    vector_store,
    retrievers=[
        cosine_retriever,
        keyword_retriever,
        beam_retriever
    ]
)

response = query_engine.query("What are the differences between Neptune Database and Neptune Analytics?")

print(response.response)

#### Reranking beam search (GPU)

The example below uses a `BGEReranker` with a `RerankingBeamGraphSearch` to rerank statements while conducting the beam search.

There will be a delay the first time this runs while the reranker downloads tensors.

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit import LexicalGraphQueryEngine
from graphrag_toolkit.storage import GraphStoreFactory
from graphrag_toolkit.storage import VectorStoreFactory
from graphrag_toolkit.retrieval.retrievers import RerankingBeamGraphSearch, StatementCosineSimilaritySearch, KeywordRankingSearch
from graphrag_toolkit.retrieval.post_processors import BGEReranker

import nest_asyncio
nest_asyncio.apply()

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

cosine_retriever = StatementCosineSimilaritySearch(
    vector_store=vector_store,
    graph_store=graph_store,
    top_k=50
)

keyword_retriever = KeywordRankingSearch(
    vector_store=vector_store,
    graph_store=graph_store,
    max_keywords=10
)

reranker = BGEReranker(
    gpu_id=0,
    batch_size=128
)

beam_retriever = RerankingBeamGraphSearch(
    vector_store=vector_store,
    graph_store=graph_store,
    reranker=reranker,
    initial_retrievers=[cosine_retriever, keyword_retriever],
    max_depth=8,
    beam_width=100
)

query_engine = LexicalGraphQueryEngine.for_semantic_guided_search(
    graph_store, 
    vector_store,
    retrievers=[
        cosine_retriever,
        keyword_retriever,
        beam_retriever
    ]
)

response = query_engine.query("What are the differences between Neptune Database and Neptune Analytics?")

print(response.response)

#### Post-processors 

The example below uses a `StatementDiversityPostProcessor`, `SentenceReranker` and `StatementEnhancementPostProcessor`.

  - `SentenceReranker` - Reranks statements using the `mixedbread-ai/mxbai-rerank-xsmall-v1` model. 

  - `StatementEnhancementPostProcessor` - Enhances statements by using chunk context and an LLM to improve content while preserving original metadata.

  - `StatementDiversityPostProcessor` - Removes similar statements using TF-IDF similarity with a default threshold of 0.975 to ensure diversity in the processed nodes.

Before running `StatementDiversityPostProcessor` for the first time, load the following package:

```
python -m spacy download en_core_web_sm
```

If you're running on a GPU device, you can replace the `SentenceReranker` with a `BGEReranker`, which reranks statements using the ``BAAI/bge-reranker-v2-minicpm-layerwise`` model.

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
%reload_ext dotenv
%dotenv

from graphrag_toolkit import LexicalGraphQueryEngine
from graphrag_toolkit.storage import GraphStoreFactory
from graphrag_toolkit.storage import VectorStoreFactory
from graphrag_toolkit.retrieval.post_processors import BGEReranker, SentenceReranker, StatementDiversityPostProcessor, StatementEnhancementPostProcessor
import os

import nest_asyncio
nest_asyncio.apply()

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

query_engine = LexicalGraphQueryEngine.for_semantic_guided_search(
    graph_store, 
    vector_store,
    post_processors=[
        SentenceReranker(), 
        StatementDiversityPostProcessor(), 
        StatementEnhancementPostProcessor()
    ]
)

response = query_engine.query("What are the differences between Neptune Database and Neptune Analytics?")

print(response.response)